In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
def open_files(date):
    focus_text = "2018-02-" + date + "_focus/2018-02-" + \
                 date + "_focus_group.json"
    krx_text = "krx_code.json"
    increase_text = "2018-02-" + date + "_opening_increase.json"
    price_text = "2018-02-" + date + "_price_and_everything.json"
    with open(focus_text, 'r', encoding='UTF-8') as focus_group:
        focus = json.load(focus_group)
    with open(krx_text, 'r', encoding='UTF-8') as krx:
        krx_code = json.load(krx)
    with open(increase_text, 'r', encoding='UTF-8') as oi:
        INCREASE = json.load(oi)
    with open(price_text, 'r', encoding='UTF-8') as oi:
        price = json.load(oi)
    
    return focus, krx_code, INCREASE, price   

In [14]:
def get_time_disc(date):
    
    time = []
    prefix = "2018-02-" + date + "_focus/discussion_2018-02-" + date
    for hour in range(9,16):
        for minute in range(0, 6):
            time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ \
                        ":" + str(minute) + "0.json")
    
    time = time[1:40]
    
    return time

In [15]:
def df_list_dis(date):
    df_list = []
    time = get_time_disc(date)
    for x in time:
        with open(x, 'r', encoding='UTF-8') as f:
            discussion = json.load(f)        
            discuss_df = pd.DataFrame(discussion, columns = ["post_num", \
                                "unique_id", "click", "like", "dislike", 
                                "name", "time"])
            reset_col = ["name", "time", "post_num", "unique_id", "click", \
                         "like", "dislike"]
            discuss_df = discuss_df[reset_col]
            df_list.append(discuss_df)
    return df_list
    

In [56]:
#half an hour low freq
'''
def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if ((ind + 1) % 3  == 0) and (ind != 2):
            df_total = df.merge(df_list[ind - 5], on = \
                                key_list).merge(df_list[ind - 4], \
                                on = key_list).merge(df_list[ind - 3], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
            total = total[total["name"].isin(INCREASE)]   
    
    return total  
'''

In [67]:
#half an hour high freq
'''
def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if ind >= 5:
            df_total = df.merge(df_list[ind - 5], on = \
                                key_list).merge(df_list[ind - 4], \
                                on = key_list).merge(df_list[ind - 3], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
            total = total[total["name"].isin(INCREASE)]   
    
    return total  
'''

In [16]:
#ten min low freq

def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if (ind % 3 == 0) and (ind != 0):
            df_total = df.merge(df_list[ind - 3], on = \
                                key_list).merge(df_list[ind - 2], \
                                on = key_list).merge(df_list[ind - 1], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
            total = total[total["name"].isin(INCREASE)]   
    
    return total 


In [20]:
#ten min high freq
'''
def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if ind >= 3:
            df_total = df.merge(df_list[ind - 3], on = \
                                key_list).merge(df_list[ind - 2], \
                                on = key_list).merge(df_list[ind - 1], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
            total = total[total["name"].isin(INCREASE)]   
    
    return total 
'''

In [17]:
COLUMN_DISC =["name", "time", "post_num", "unique_id", "click", \
              "like", "dislike", "time_1", "post_num_1", "unique_id_1", "click_1", \
              "like_1", "dislike_1", "time_2", "post_num_2", "unique_id_2", "click_2", \
              "like_2", "dislike_2", "time_3", "post_num_3", "unique_id_3", "click_3", \
              "like_3", "dislike_3"]

def get_discuss_df(date):
    discussion_list = df_list_dis(date)
    discuss_df = list_to_df(date, discussion_list, COLUMN_DISC, 
                            ['name']).reset_index().drop(["index"], axis = 1)
    return discuss_df

In [18]:
def df_list_price(date):
    price_df = pd.DataFrame(price, columns = ["index", "code", "name", "time", "price", \
                                         "price_dif", "sell", "buy", "volume", "variation"])
    price_df = price_df[["code", "name", "time", "price", \
                    "price_dif", "sell", "buy", "volume", "variation"]][1:]
    text = "2018-02-" + date + " 09:00"
    price_df = price_df[price_df["time"] != text]
    time_list = price_df['time'].unique().tolist()
    price_df_list = []
    for time in time_list:
        df = price_df[price_df["time"] == time]
        df = df[["code", "name", "time", "price", \
                "price_dif", "sell", "buy", "volume", "variation"]]
        price_df_list.append(df)
        
    return price_df_list

In [19]:
COLUMN_PRICE=["code", "name", "time", "price", "price_dif", "sell", 
              "buy", "volume", "variation", "time_1", "price_1", 
              "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1",
              "time_2", "price_2", "price_dif_2", "sell_2", "buy_2", 
              "volume_2", "variation_2", "time_3", "price_3", 
              "price_dif_3", "sell_3", "buy_3", "volume_3", "variation_3"]

def get_price_df(date):
    price_df_list = df_list_price(date)
    price_df = list_to_df(date, price_df_list, COLUMN_PRICE, \
                          ['code', 'name']).reset_index().drop(["index"], axis = 1)
    
    return price_df

In [20]:
COLUMN_TOTAL = ['name', 'code', 'time', 'price', 'time_1', "price_1", \
                "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1", \
                'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', \
                'time_2', 'price_2', "price_dif_2", "sell_2", "buy_2", \
                "volume_2", "variation_2", 'post_num_2', 'unique_id_2', 'click_2', \
                'like_2', 'dislike_2', 'time_3', 'price_3', "price_dif_3", \
                "sell_3", "buy_3", "volume_3", "variation_3", 'post_num_3', \
                'unique_id_3', 'click_3', 'like_3', 'dislike_3']

def get_total_df(date):
    price_df = get_price_df(date)
    discuss_df = get_discuss_df(date)
    total_df = pd.merge(price_df, discuss_df, on = ['name', \
                                                    'time', 'time_1', 'time_2', 'time_3'])
    total_df = total_df[COLUMN_TOTAL]
    
    return total_df

In [21]:
import numpy as np
import re

with open("company_size.json", 'r', encoding='UTF-8') as f:
    company_size = json.load(f)

company_df = pd.DataFrame({"company_name": np.zeros(0), \
                           "code": np.zeros(0), "market": np.zeros(0), 
                          "company_size": np.zeros(0)})

company_df = pd.DataFrame(company_size, columns = ["name", "code", "market", "size"])

KOSPI = []
KOSDAQ = []
TRASH = []
MKT_CAP = {}

for idx, row in company_df.iterrows():
    if re.sub('[0-9 ,위]', '', row["market"]) == "코스피":
        KOSPI.append(row["code"])
    elif re.sub('[0-9 ,위]', '', row["market"]) == "코스닥":
        KOSDAQ.append(row["code"])
    else:
        TRASH.append(row["code"])
    
    MKT_CAP[row["code"]] = int(re.sub('[조억원,]', '', row["size"]))

In [22]:
def add_company(date):
    
    total = get_total_df(date)
    total["mkt_cap"] = np.nan
    total["kospi"] = np.nan
    total["kosdaq"] = np.nan
    total["trash"] = np.nan
    
    for index, row in total.iterrows():
        mkt_cap = MKT_CAP[row["code"]]
        if row["code"] in KOSPI:
            kospi = 1
        else:
            kospi = 0

        if row["code"] in KOSDAQ:
            kosdaq = 1
        else:
            kosdaq = 0

        if row["code"] in TRASH:
            trash = 1
        else:
            trash = 0
        total.set_value(index,'mkt_cap', mkt_cap)
        total.set_value(index,'kospi', kospi)
        total.set_value(index,'kosdaq', kosdaq)
        total.set_value(index,'trash', trash)  
    
    return total


In [23]:
def transform_df(date):
    
    total = add_company(date)   
    var_to_transform = ['price', 'price_1', 'price_dif_1', 'sell_1', 
                        'buy_1', 'volume_1', 'variation_1', 'price_2', 
                        'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 
                        'variation_2', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 
                        'volume_3', 'variation_3']
    for index, row in total.iterrows():
        for var in var_to_transform:
            if isinstance(row[var], str):

                data = row[var].split(",")
                value = ''.join(data)
                value = int(value)
                total.set_value(index, var, value)
    return total
    

In [13]:
focus, krx_code, INCREASE, price = open_files('14')
total_14 = transform_df('14')
focus, krx_code, INCREASE, price = open_files('20')
total_20 = transform_df('20')
focus, krx_code, INCREASE, price = open_files('21')
total_21 = transform_df('21')
focus, krx_code, INCREASE, price = open_files('22')
total_22 = transform_df('22')

In [24]:
focus, krx_code, INCREASE, price = open_files('23')
total_23 = transform_df('23')

In [25]:
total_df = pd.concat([total_14, total_20, total_21, total_22, total_23])

In [26]:
total_df = total_df.reset_index().drop(["index"], axis = 1)

In [27]:
total_df["yesterday_closing_price"] = total_df["price_1"] - total_df["price_dif_1"]

In [28]:
total_df["is_maximum"] = (((total_df["price_1"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5) | \
(((total_df["price_2"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5) | \
(((total_df["price_3"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5)
total_df["is_maximum"] = total_df["is_maximum"].astype(int)

In [29]:
total_df["is_minimum"] = (((total_df["price_1"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5) | \
(((total_df["price_2"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5) | \
(((total_df["price_3"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5)
total_df["is_minimum"] = total_df["is_maximum"].astype(int)

In [30]:
total_df["price_volatility"] = (((total_df[["price_1", "price_2", "price_3"]].max(axis=1)) / (total_df[["price_1", "price_2", "price_3"]].min(axis=1))) - 1) * 100

In [31]:
total_df["price_trend"] = (((total_df["price_1"] - total_df["price_2"]) < 0).astype(int) + ((total_df["price_2"] - total_df["price_3"]) < 0).astype(int)) - (((total_df["price_1"] - total_df["price_2"]) > 0).astype(int) + ((total_df["price_2"] - total_df["price_3"]) > 0).astype(int))

In [32]:
total_df["average_price_volatility"] = total_df["price_trend"] * total_df["price_volatility"] / 2

In [33]:
total_df["sell_minus_buy_1"] = total_df["sell_1"] - total_df["buy_1"]

In [34]:
total_df["sell_minus_buy_2"] = total_df["sell_2"] - total_df["buy_2"]

In [35]:
total_df["sell_minus_buy_3"] = total_df["sell_3"] - total_df["buy_3"]

In [36]:
total_df["is_price_gap_stable"] = ((total_df["sell_minus_buy_1"] == total_df["sell_minus_buy_2"]) & (total_df["sell_minus_buy_2"] == total_df["sell_minus_buy_3"])).astype(int)

In [37]:
total_df["price_gap_volatility"] = (((total_df[["sell_minus_buy_1", "sell_minus_buy_2", "sell_minus_buy_3"]].max(axis=1)) / (total_df[["sell_minus_buy_1", "sell_minus_buy_2", "sell_minus_buy_3"]].min(axis=1)))) - 1

In [38]:
total_df["is_like_higher"] = (total_df["like_3"] > total_df["dislike_3"]).astype(int)

In [39]:
total_df["volume_trend"] = ((((total_df["volume_3"]) - (total_df["volume_2"] * 1+1e-3)) \
 / ((total_df["volume_2"]) - (total_df["volume_1"] * 1-1e-4))) - 1) * 100

In [40]:
total_df["post_num_trend"] = ((((total_df["post_num_3"]) - (total_df["post_num_2"] * 1+1e-3)) \
 / ((total_df["post_num_2"]) - (total_df["post_num_1"] * 1-1e-4))) - 1) * 100

In [41]:
total_df["unique_id_trend"] = ((((total_df["unique_id_3"]) - (total_df["unique_id_2"] * 1+1e-3)) \
 / ((total_df["unique_id_2"]) - (total_df["unique_id_1"] * 1-1e-4))) - 1) * 100

In [42]:
total_df["click_trend"] = ((((total_df["click_3"]) - (total_df["click_2"] * 1+1e-3)) \
 / ((total_df["click_2"]) - (total_df["click_1"] * 1-1e-4))) - 1) * 100

In [43]:
total_df["price_increase"] = ((total_df["price"] / total_df["price_3"]) - 1) * 100

In [44]:
total_df["did_price_increase"] = (total_df["price_increase"] > 0).astype(int)

In [47]:
total_df.to_json('ten_min_low_freq_no_cheating.json', orient='values')

In [46]:
total_df

,name,code,time,price,time_1,price_1,price_dif_1,sell_1,buy_1,volume_1,...,sell_minus_buy_3,is_price_gap_stable,price_gap_volatility,is_like_higher,volume_trend,post_num_trend,unique_id_trend,click_trend,price_increase,did_price_increase
0,호텔신라,008770,2018-02-14 09:40,83900,2018-02-14 09:10,84700,1800,84800,84700,29336,...,100,1,0.000000,1,72.8395,-100.1,-100.1,-43.7191,0,0
1,대한광통신,010170,2018-02-14 09:40,7090,2018-02-14 09:10,7060,90,7060,7050,101713,...,10,1,0.000000,0,28.1127,2.9989e+06,2.9989e+06,1.41999e+08,0.141243,1
2,포스코 ICT,022100,2018-02-14 09:40,8350,2018-02-14 09:10,8310,0,8330,8310,44311,...,10,0,1.000000,0,-47.3209,998900,998900,9.64815,1.08959,1
3,한화케미칼,009830,2018-02-14 09:40,32500,2018-02-14 09:10,32700,550,32700,32650,32647,...,100,0,1.000000,0,120.857,199.87,199.87,42.741,-0.15361,0
4,지엘팜텍,204840,2018-02-14 09:40,4735,2018-02-14 09:10,4680,50,4700,4680,114565,...,10,0,1.500000,0,-9.39576,-100.1,-100.1,-21.5058,0.959488,1
5,코웰패션,033290,2018-02-14 09:40,5420,2018-02-14 09:10,5270,90,5280,5270,22780,...,20,0,1.000000,1,-30.0755,-1100,-1100,24.9953,1.8797,1
6,대한항공,003490,2018-02-14 09:40,35650,2018-02-14 09:10,35700,300,35750,35700,7560,...,50,1,0.000000,0,-63.512,-0.109989,-0.109989,168.414,0,0
7,대한뉴팜,054670,2018-02-14 09:40,13800,2018-02-14 09:10,13800,0,13800,13750,2556,...,100,0,1.000000,0,-17.6742,-1100,-1100,-9.68094,0.729927,1
8,레드캡투어,038390,2018-02-14 09:40,15350,2018-02-14 09:10,15350,100,15350,15200,7,...,150,1,0.000000,0,-0.109989,-1100,-1100,-1100,0,0
9,이마트,139480,2018-02-14 09:40,288500,2018-02-14 09:10,283500,5000,284000,283500,8469,...,500,1,0.000000,1,6.98242,-100.1,-100.1,183.312,0.347826,1
